UNICEF Water vs. GDP vs. Cholera Mortality Rate

In [1]:
#importing dependencies
import pandas as pd 
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from config import password

UNICEF Dataset - clean & convert to dataframe

In [2]:
#loading UNICEF file
csv_file = "../Project-2/drink_water.csv"
unicef_df = pd.read_csv(csv_file, encoding="ISO-8859-1")
unicef_df.head()

,Country_Code,Country,Years,Total improved,Piped on premises,Other improved,Other unimproved,Surface water
0,AFG,Afghanistan,1990,0,1,0,0,0
1,AFG,Afghanistan,2015,55,12,43,39,6
2,ALB,Albania,1990,0,0,0,0,0
3,ALB,Albania,2015,95,82,13,5,0
4,DZA,Algeria,1990,91,67,24,8,1


In [3]:
#only pull 2015 years
unicef2015_df = unicef_df.loc[unicef_df["Years"] == 2015.0, :]

In [4]:
#remove decimal from years
pd.options.display.float_format = '{:.0f}'.format
unicef2015_df.head()

,Country_Code,Country,Years,Total improved,Piped on premises,Other improved,Other unimproved,Surface water
1,AFG,Afghanistan,2015,55,12,43,39,6
3,ALB,Albania,2015,95,82,13,5,0
5,DZA,Algeria,2015,84,77,7,16,0
7,ASM,American Samoa,2015,100,92,8,0,0
9,AND,Andorra,2015,100,100,0,0,0


GDP Dataset - transform & convert to dataframe

In [5]:
#import GDP from mysql
engine = create_engine("mysql://root:" + password + "@localhost:3306/project2")
conn = engine.connect()

In [6]:
#drop all columns except 2015
gdp_data = pd.read_sql("SELECT * FROM `Country_GDP_v4`", conn)

In [7]:
gdp_data = gdp_data.rename(index=str, columns={"Country Code":"Country_Code", "2015":"2015_gdp"})
gdp_data.head()

,﻿Country Name,Country_Code,2015_gdp
0,Aruba,ABW,
1,Andorra,AND,
2,Afghanistan,AFG,19199437989
3,Angola,AGO,1.03E+11
4,Albania,ALB,11455595709


In [8]:
# Merge the two DataFrames together based on the Country Code
country_df = pd.merge(unicef2015_df, gdp_data, on="Country_Code")
type(country_df)

pandas.core.frame.DataFrame

In [9]:
country_df['2015_gdp'] = pd.to_numeric(country_df['2015_gdp'])
country_df['Total improved'] = pd.to_numeric(country_df['Total improved'])

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

country_df.plot(kind="scatter", x="2015_gdp", y="Total improved", grid=True, figsize=(5,5),
              title="GDP vs. Total Improved Drinking Water")
plt.xlabel("2015 GDP ($USD)")
plt.ylabel("Total Improved")
plt.savefig("GDP vs. Improved.png")

In [ ]:
country_df.to_csv("Combined_data.csv")